In [1]:
import pandas as pd
import pickle
from scipy import stats

In [2]:
dfAgg = pd.read_pickle('group-sentence-data-2.pkl')
dfAgg

,Sentence,all,NEU,HAP,SAD,ANG,FEA,DIS,Caucasian,Caucasian-NEU,...,Female-Asian-ANG,Female-Asian-FEA,Female-Asian-DIS,Female-Unknown,Female-Unknown-NEU,Female-Unknown-HAP,Female-Unknown-SAD,Female-Unknown-ANG,Female-Unknown-FEA,Female-Unknown-DIS
0,IEO,0.520987,0.392337,0.469703,0.644246,0.399665,0.594038,0.540165,0.515709,0.399193,...,0.421533,0.901983,0.637250,0.643375,0.35290,0.352900,0.823533,0.372533,0.941200,0.823533
1,TIE,0.048354,0.045610,0.036395,0.063675,0.042457,0.059503,0.042457,0.045174,0.034500,...,0.034500,0.034500,0.155200,0.040250,0.03450,0.034500,0.034500,0.034500,0.069000,0.034500
2,IOM,0.045886,0.037000,0.039443,0.050024,0.039035,0.057757,0.052058,0.047423,0.037000,...,0.037000,0.037000,0.037000,0.037000,0.03700,0.037000,0.037000,0.037000,0.037000,0.037000
3,IWW,0.086049,0.037000,0.073635,0.104160,0.043920,0.142832,0.114747,0.098836,0.037000,...,0.037000,0.037000,0.074050,0.178983,0.03700,0.037000,0.037000,0.037000,0.888900,0.037000
4,TAI,0.094873,0.043107,0.074455,0.125334,0.072414,0.163597,0.090330,0.096210,0.046110,...,0.037000,0.111100,0.111100,0.086400,0.03700,0.185200,0.185200,0.037000,0.037000,0.037000
5,MTI,0.062623,0.038856,0.060709,0.079970,0.048116,0.078118,0.069969,0.068765,0.041633,...,0.083300,0.033300,0.033300,0.033300,0.03330,0.033300,0.033300,0.033300,0.033300,0.033300
6,IWL,0.064623,0.038796,0.057845,0.079460,0.038064,0.097412,0.076163,0.067913,0.036033,...,0.033300,0.350000,0.033300,0.033300,0.03330,0.033300,0.033300,0.033300,0.033300,0.033300
7,ITH,0.057220,0.028417,0.049402,0.071004,0.029650,0.095079,0.069769,0.063923,0.027800,...,0.027800,0.138900,0.027800,0.037050,0.02780,0.027800,0.083300,0.027800,0.027800,0.027800
8,DFA,0.014913,0.005233,0.013082,0.020931,0.003138,0.016221,0.030870,0.016652,0.001561,...,0.000000,0.000000,0.000000,0.000000,0.00000,0.000000,0.000000,0.000000,0.000000,0.000000
9,ITS,0.093564,0.050381,0.083580,0.103205,0.050610,0.151480,0.121651,0.104292,0.052962,...,0.071400,0.142850,0.035700,0.047600,0.10710,0.035700,0.035700,0.035700,0.035700,0.035700


In [3]:
# Finds maximum pairwise mean WER difference between sentences for all speakers
sentences = dfAgg['Sentence']
wers = list(dfAgg['all'][1:-1])
maxVal = 0
series = ['nan','nan']

for i in range(len(wers)):
    for j in range(len(wers)):
        diff = abs(wers[i] - wers[j])
        if diff > maxVal:
            maxVal = diff
            series[0] = sentences[i]
            series[1] = sentences[j]
print(maxVal)
print(series)

0.13344102564102564
['ITH', 'ITS']


In [4]:
# Finds sentence with maximum mean WER difference for groups a and b
sentences = dfAgg['Sentence']
a = list(dfAgg['African American'][1:-1])
b = list(dfAgg['Caucasian'][1:-1])
maxVal = 0

for i in range(len(wers)):
    diff = abs(a[i]-b[i])
    if diff > maxVal:
        maxVal = diff
        series = sentences[i]
    
print(maxVal)
print(series)

0.03974369100844509
ITS


In [5]:
# Finds sentence with maximum mean WER difference for groups a and b
sentences = dfAgg['Sentence']
a = list(dfAgg['Male'][1:-1])
b = list(dfAgg['Female'][1:-1])
maxVal = 0

for i in range(len(wers)):
    diff = abs(a[i]-b[i])
    if diff > maxVal:
        maxVal = diff
        series = sentences[i]
    
print(maxVal)
print(series)

0.03472428133074934
ITS


In [6]:
# Computes basic p value using a t-test
def p_val(df, feature, group):
    series1 = df.loc[df[feature]==group[0]]["WER"]
    series2 = df.loc[df[feature]==group[1]]["WER"]
    pval = stats.ttest_ind(series1, series2).pvalue
    return pval

In [7]:
df = pd.read_pickle('crema-processed-dataset-with-WER-PER.pkl')
wers = list(df['WER'])
wers = [float(wer[:-1])*.01 for wer in wers]
df['WER'] = wers

In [8]:
print(p_val(df, "Race", ["Caucasian", "African American"]))
print(p_val(df, "Sex", ["Male", "Female"]))

0.012421655719682952
0.11113379660221795
